In [38]:
## merge subject ids in the test set with demographic info.

## updates on Jun 28:
## add MEM and EF (for ML predition) score to subject info.
## use baseline fMRI features to predict further MEM and EF scores.

import pandas as pd
import os
import re

input_file   = 'adni_out01_time_signal_test_train_split/test_set_roi_signals.csv'
input_file2  = 'adni_subject_info/MRI_information_All_Measurement.csv'
input_file3  = 'adni_subject_info/ADNI_MEM_EF_Composit_Score.csv'

output_dir   = 'adni_out02_test_subject_info'

file_test    = pd.read_csv(input_file, header = None)
file_test    = file_test[0].tolist()
subject_test = set()

subject_id = []
ses_id = []

for f in file_test:
    subject_id += [re.search('(.*)_sub-(.*)-ses-(.*).txt', f).group(2)]
    ses_id +=  [re.search('(.*)_sub-(.*)-ses-(.*).txt', f).group(3)]

subject_info = pd.DataFrame({'subject_id': subject_id, 'ses_id': ses_id})


In [39]:
subject_info

,subject_id,ses_id
0,002_S_0295,2012-05-10_15_42_37.0
1,002_S_0413,2017-06-21_13_23_38.0
2,002_S_0685,2011-07-08_07_04_27.0
3,002_S_0729,2012-08-07_07_21_09.0
4,002_S_1155,2017-04-24_13_21_32.0
...,...,...
324,941_S_4100,2017-10-27_11_05_51.0
325,941_S_4187,2017-06-05_14_05_55.0
326,941_S_4292,2017-08-21_15_05_09.0
327,941_S_4365,2017-08-28_14_06_46.0


In [40]:
demographic_data = pd.read_csv(input_file2)
g = demographic_data.groupby(['PTID']).first().reset_index()
df_merge = subject_info.merge(g[['PTID', 'AGE', 'PTGENDER', 'DX_bl', 'DX']], how = 'left', 
                              left_on = 'subject_id', right_on = 'PTID').drop(columns = 'PTID')
df_merge

,subject_id,ses_id,AGE,PTGENDER,DX_bl,DX
0,002_S_0295,2012-05-10_15_42_37.0,84.8,Male,CN,CN
1,002_S_0413,2017-06-21_13_23_38.0,76.3,Female,CN,CN
2,002_S_0685,2011-07-08_07_04_27.0,89.6,Female,CN,CN
3,002_S_0729,2012-08-07_07_21_09.0,65.1,Female,LMCI,LMCI
4,002_S_1155,2017-04-24_13_21_32.0,57.8,Male,LMCI,LMCI
...,...,...,...,...,...,...
324,941_S_4100,2017-10-27_11_05_51.0,78.5,Female,CN,CN
325,941_S_4187,2017-06-05_14_05_55.0,62.0,Male,LMCI,LMCI
326,941_S_4292,2017-08-21_15_05_09.0,70.9,Male,CN,CN
327,941_S_4365,2017-08-28_14_06_46.0,80.3,Male,CN,CN


In [41]:
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

df_merge.to_csv(output_dir + '/subject_info_merge.csv')

In [42]:

composite_score = pd.read_csv(input_file3)
composite_score = composite_score.dropna(subset=['ADNI_MEM', 'ADNI_EF'])
composite_score = composite_score.sort_values('EXAMDATE', ascending=False).drop_duplicates(['PTID'])
composite_score.sort_values('PTID')

,RID,PTID,VISCODE,EXAMDATE,DX_bl,DX,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,ADNI_MEM,ADNI_EF
12,295,002_S_0295,72,20120510,CN,CN,84.8,Male,18,Not Hisp/Latino,White,Married,1.0,0.953,0.209
31,413,002_S_0413,132,20170615,CN,CN,76.3,Female,16,Not Hisp/Latino,White,Married,0.0,1.574,2.204
39,559,002_S_0559,36,20090702,CN,CN,79.3,Male,16,Not Hisp/Latino,White,Widowed,1.0,0.868,0.872
44,619,002_S_0619,24,20080807,AD,AD,77.5,Male,12,Not Hisp/Latino,White,Married,2.0,-1.357,-1.595
57,685,002_S_0685,72,20120727,CN,CN,89.6,Female,16,Not Hisp/Latino,White,Married,0.0,0.876,0.741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13768,6570,941_S_6570,0,20180910,CN,CN,71.2,Male,18,Not Hisp/Latino,White,Married,NaN,0.493,2.139
13769,6574,941_S_6574,0,20180918,CN,CN,73.5,Female,16,Not Hisp/Latino,White,Married,NaN,0.476,0.763
13770,6575,941_S_6575,0,20180918,CN,CN,73.5,Male,20,Not Hisp/Latino,White,Married,NaN,0.483,1.533
13771,6580,941_S_6580,0,20180914,SMC,SMC,79.9,Female,18,Not Hisp/Latino,White,Divorced,NaN,0.127,-1.023


In [46]:
df_merge2 = df_merge.merge(composite_score[['PTID', 'EXAMDATE', 'ADNI_MEM', 'ADNI_EF']], how = 'left', 
                              left_on = 'subject_id', right_on = 'PTID').drop(columns = 'PTID')
df_merge2

,subject_id,ses_id,AGE,PTGENDER,DX_bl,DX,EXAMDATE,ADNI_MEM,ADNI_EF
0,002_S_0295,2012-05-10_15_42_37.0,84.8,Male,CN,CN,20120510.0,0.953,0.209
1,002_S_0413,2017-06-21_13_23_38.0,76.3,Female,CN,CN,20170615.0,1.574,2.204
2,002_S_0685,2011-07-08_07_04_27.0,89.6,Female,CN,CN,20120727.0,0.876,0.741
3,002_S_0729,2012-08-07_07_21_09.0,65.1,Female,LMCI,LMCI,20130808.0,-0.899,-0.562
4,002_S_1155,2017-04-24_13_21_32.0,57.8,Male,LMCI,LMCI,20180508.0,0.825,1.753
...,...,...,...,...,...,...,...,...,...
324,941_S_4100,2017-10-27_11_05_51.0,78.5,Female,CN,CN,20171026.0,-0.420,-0.534
325,941_S_4187,2017-06-05_14_05_55.0,62.0,Male,LMCI,LMCI,20170605.0,0.426,0.120
326,941_S_4292,2017-08-21_15_05_09.0,70.9,Male,CN,CN,20170810.0,0.711,1.167
327,941_S_4365,2017-08-28_14_06_46.0,80.3,Male,CN,CN,20180828.0,-0.715,-1.228


In [45]:
df_merge2.to_csv(output_dir + '/subject_info_merge2.csv')